In [ ]:
import datetime as dt
import sys

import xlwings as xw
import numpy as np
import pandas as pd

In [ ]:
xw.__version__

# 基本

## `xw.view()`: Excel as viewer for tabular data

In [ ]:
data = np.random.rand(100, 100)
data

In [ ]:
# Open a new book
xw.view(data)

In [ ]:
# Reuse an existing sheet(sheets gets cleared with every call)
xw.view(np.random.rand(5, 5), xw.sheets.active)

In [ ]:
xw.view(np.random.rand(3, 3), xw.sheets.active)

## Connect to a Book

In [ ]:
# Fire up a new book in the active Excel instance
wb1 = xw.Book()

In [ ]:
# Connect to an unsaved book(Looks in all Excel instance)
wb1 = xw.Book("Book2")

In [ ]:
# Connect to a book by file name or full path and opens it
wb1 = xw.Book(r"C:\Users\XXX\Desktop\Myworkbook.xlsx")

## Sheet Object

In [ ]:
sheet = wb1.sheets[0]

## The Range Object

In [ ]:
# Write Value
sheet.range("A1").value = "hello xlwings!"

In [ ]:
# Read value
sheet.range("A1").value

In [ ]:
# Write the same value to multiple cells
sheet.range("A3:B4").value = 123

In [ ]:
# Excel's numerical format is float
sheet.range("A3").value

In [ ]:
# Datetime
sheet.range("A6") = dt.datetime(2020, 12, 25, 22, 22, 22)
sheet.range("A6").value

In [ ]:
# Index natation (1-Based like Excel)
sheet.range(1, 1).value

In [ ]:
# Formula
sheet.range("B1").formula = "=SUM(A3:B4)"

In [ ]:
# Named ranges
sheet.range("B1").name = "test"
sheet.range("test").formula

In [ ]:
foo = sheet.range("test").value
foo

In [ ]:
sheet.range("test").value = "Likethis"

## 2d Ranges

In [ ]:
sheet.range("A3:B4").value

In [ ]:
# Index notation
sheet.range((3, 1), (4, 2)).value

In [ ]:
# Assign a nested list to the top-left corner
sheet.range("A9").value = [["a string", 1, 2, 3],
                           [dt.date(2021, 1, 1), 123.5, None, None]]

In [ ]:
# Range expansion: "table", "down", "right"
# Correspond to Ctrl+Shift+Down and/or right
# They return a Range object!
sheet.range("A9").expand("table")

In [ ]:
# "table" is default
sheet.range("A9").expand.value

In [ ]:
# Use .clear to also clear the formatting
sheet.range("A9").expand().clear_contents()

## 1d vectors

In [ ]:
# Horizontal
sheet.range("A12").value = [1, 2, 3, 4]

In [ ]:
# Vertical
sheet.range("A13").options(transpose=True).value = [5, 6, 7, 8]
# this is the same as:
# sheet.range("A13").value = [[5], [6], [7], [8]]

In [ ]:
sheet.range("A12").expand("right").value

In [ ]:
sheet.range("A12").expand("down").value

## ndim

In [ ]:
sheet.range("A12").options(ndim=2, expand="right").value

In [ ]:
sheet.range("A12").options(ndim=2, expand="down").value

## Autofit

In [ ]:
# autofit columns and rows based on a single cell
sheet.range("A3").autofit()

In [ ]:
# autofit columns based on range
sheet.range("A1:C3").columns.autofit()

In [ ]:
# autofit a whole column
sheet.range("A:A").autofit()

## Background color

In [ ]:
# Assign an RGB value
sheet.range("A1").color = (0, 255, 0)
sheet.range("A1").color

## Range indexing/slicing

In [ ]:
rng = sheet.range("A1:D5")
rng[0, 0]

In [ ]:
rng[1]

In [ ]:
rng[:, 3:]

In [ ]:
rng[1:3, 1:3]

In [ ]:
xw.books.active.close()

## Full qualification

In [ ]:
# Get all available PIDs (Process IDs)
xw.apps.keys()

In [ ]:
# This allows us to specify a specific Excel instance
pid = xw.apps.keys()[0] # or xw.apps.active.pid

In [ ]:
xw.apps[pid].books[0].sheets[0].range("A1")

In [ ]:
xw.apps(pid).books(1).sheets(1).range("A1")

In [ ]:
# Instead of indices we can also use names
xw.apps[pid].books["Book1"].sheets["Sheet1"].range("A1")
xw.apps(pid).books("Book1").sheets("Sheet1").range("A1")

## work with multiple apps

In [ ]:
app1 = xw.apps[pid]
app2 = xw.App()

In [ ]:
# Open the same workbook twice in different Excel instances
app1.books.open("foo.xlsx")
app2.books.open("foo.xlsx")

In [ ]:
# this will throw an error
# xw.Book("foo.xlsx")

In [ ]:
# The following syntax is required
# if the same file is opened in more than 1 instance
print(app1.books["foo.xlsm"])
print(app2.books["foo.xlsm"])
print(app1.books["foo.xlsm"].app)
print(app1.books["foo.xlsm"].app)

## Active Objects

In [ ]:
# Active app
xw.apps.active

In [ ]:
# active book in active app
xw.books.active

In [ ]:
# active sheet in active book in active app
xw.sheets.active

In [ ]:
# This is a special shortcut for interactive use only
# It takes the active sheet form the active book
xw.Range("A1").value

In [ ]:
app2.kill()

## Sheets

In [ ]:
xw.sheets[0].name

In [ ]:
xw.sheets.count # same as len(xw.sheets)

In [ ]:
xw.sheets.add(name="New Sheet By xlwings", after="Sheet1")

In [ ]:
sheet = xw.sheets[0]
sheet["A1"]

In [ ]:
sheet["A1:B5"]

In [ ]:
sheet[0, 1]

In [ ]:
sheet[:10, :10]

## Charts

In [ ]:
wb = xw.Book()
sheet = wb.sheets[0]
sheet.range("A1").value = [["one", "two"],
                           [1.1, 2.2],
                           [3.3, None]]

In [ ]:
chart = sheet.charts.add()
chart.set_source_data(sheet.range("A1").expand())
chart.chart_type = "line"
chart.top = sheet.range("A5").top

In [ ]:
chart.chart_type = "area"

In [ ]:
xw.constants.chart_types

In [ ]:
wb.close()

## Matplotlib

In [ ]:
% matplotlib inline
from scipy.interpolate import interp1d
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
years = [1, 2, 3, 4, 5, 7, 10, 30]
swap_rate = np.random.rand(8)
years_new = np.linspace(1, 30, num=30)
interpolate = interp1d(years, swap_rate, kind="quadratic")

In [ ]:
fig = plt.figure(figsize=(6, 4))
swap_rate_plot = plt.plot(years, swap_rate, "o",
                          years_new, interpolate(years_new),                            "-")

In [ ]:
wb = xw.Book()
sheet = wb.sheets[0]

In [ ]:
plot = sheet.pictures.add(fig, name="SwapRate", update=True)

In [ ]:
width, height = fig.get_size_inches()
dpi = fig.get_dpi()
sheet.pictures.add(fig, name="SwapRate2", update=True,
                   left=sheet.range("A25").left, top=sheet.range("A25"),
                   width=width * dpi / 2, height=height * dpi / 2)

In [ ]:
plot.height = plt.height / 2
plot.width = plot.width / 2

In [ ]:
wb.close()

## Table objects

In [ ]:
wb = xw.Book("foo.xlsx")
sheet = wb.sheets[0]

In [ ]:
sheet.range("Table1").value

In [ ]:
sheet.range("Table1[Symbol]").value

In [ ]:
sheet.range("Table1[#All], [Last]]").value

In [ ]:
sheet.range("Table1[[#Headers], [Last]]").value

In [ ]:
sheet.range("Table1[[#Totals], [Last]]").value

In [ ]:
sheet.range("Table1[[Index]:[Last]]").value

In [ ]:
wb.close()

## Efficiency

In [ ]:
wb = xw.Book()
sheet = wb.sheets[0]

In [ ]:
sheet.range("A1").value = np.arange(5 * 30).reshape((30, 5))

In [ ]:
wb.close()

## Work around for missing features: Manipulate the underlying object

In [ ]:
wb = xw.Book()
sheet = wb.sheets[0]

In [ ]:
# On windows, the underlying object is a pywin32 COM object
# On Mac, it is an appscript object
sheet.range("A1").api

In [ ]:
if sys.platform.startswith("darwin"):
    sheet.range("A10").api.clear_formats()
elif sys.platform.startswith("win"):
    sheet.range("A10").api.ClearFormats()